In [20]:
import pandas as pd
import numpy as np
import os
import json
import sys


original_cwd = os.getcwd()

backend_path = os.path.abspath(os.path.join(original_cwd, "../.."))
print(backend_path)
print(sys.path)

if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

from shared.snowflake.client import SnowflakeClient
from experiments.utils.llm import build_prompt, get_llm_response, compare_ground_truth_vs_llm, count_number_tokens, split_docs_recursively

print(f"Returned to original working directory: {os.getcwd()}")

c:\Users\hamma\Desktop\project\NutriRAG
['c:\\Users\\hamma\\Desktop\\project\\NutriRAG\\backend', 'c:\\Users\\hamma\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'c:\\Users\\hamma\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'c:\\Users\\hamma\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'c:\\Users\\hamma\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\hamma\\AppData\\Roaming\\Python\\Python311\\site-packages', 'c:\\Users\\hamma\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages']
Returned to original working directory: c:\Users\hamma\Desktop\project\NutriRAG\backend\experiments


**load the config file**

In [21]:
if not os.path.exists("config"):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

CONFIG_FILE_PATH = "config/base_config.json"

with open(CONFIG_FILE_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

**set the experience id**

In [22]:
EXPERIENCE_ID = config["experiments_specifique_params"]["experiment_id"]

print(EXPERIENCE_ID)

1


**load the embedding file**

In [23]:
INPUT_DATA_FILE = config["input_recipies_file"]

df_recipes_cleaned = pd.read_csv(INPUT_DATA_FILE)

df_recipes_cleaned.head()

,ID,NAME_CLEAND,INGREDIENTS_CLEAND,STEPS_CLEAND,DESCRIPTION_CLEAND
0,421766,recipe name: stuffed mussels in spicy tomato s...,"recipe ingredients: mussels, ground beef, crus...","recipe steps: beard and clean mussels, open mu...",recipe description: stuffed mussels is a favor...
1,148350,recipe name: bob s meat.,"recipe ingredients: puff pastry, bacon, onion,...",recipe steps: remove package of puff pastry fr...,recipe description: this is a wonderfully tast...
2,351663,recipe name: grilled turkey breast.,"recipe ingredients: cider vinegar, garlic, gro...","recipe steps: in a medium bowl, combine the ci...",recipe description: source cooking with the di...
3,45766,recipe name: raisin banana bread.,"recipe ingredients: water, bread flour, banana...","recipe steps: measure carefully, placing all i...",recipe description: sounds like a winner to me...
4,163701,recipe name: lime pound cake 1968.,"recipe ingredients: butter, granulated sugar, ...","recipe steps: grease a ten inch tube pan, line...",recipe description: this pound cake takes baki...


## evaluation the LLM and the prompt system using the ground truth ##

**load the test dataset**

In [24]:
QUERY_TEST_FILE_PATH = config["query_test_file_path"]

with open(QUERY_TEST_FILE_PATH, "r", encoding="utf-8") as f:
    ground_truth_results = json.load(f)

print(QUERY_TEST_FILE_PATH)

data/query/query_test.json


**load the prompt**

In [25]:
PROMPT_EVAL_PATH = config["eval_prompt_file"]

with open(PROMPT_EVAL_PATH, "r") as f:
    prompt_template = f.read()

**calculate the number of tokens in the prompt template**

In [26]:
LLM_MODEL = config["llm_model"]
LLM_MODEL_CONTEXT_WINDOWS = config["context_window"]
LLM_TEMPERATURE = config["temperature"]
LLM_MAX_TOKENS = config['max_tokens']

num_token_prompt_template = count_number_tokens(client=SnowflakeClient(), model=LLM_MODEL, text=str(prompt_template))
num_token_prompt_template

449

**define the schema of the response**

In [27]:
# JSON schema for recipe relevance ratings
json_schema = config['llm_json_format']
json_schema

{'type': 'object',
 'properties': {'query_text': {'type': 'string'},
  'relevance_judgments': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'doc_id': {'type': 'integer'},
     'relevance_score': {'type': 'number'},
     'justification': {'type': 'string'}},
    'required': ['doc_id', 'relevance_score', 'justification']}}},
 'required': ['query_text', 'relevance_judgments']}

In [ ]:

import json

COLUMNS_TEXT = config["columns_to_clean"]

llm_results = []

for query in ground_truth_results:
    query_text = query["query_text"]
    
    num_token_doc = count_number_tokens(client=SnowflakeClient(), model=LLM_MODEL, text=str(query))
    
    # Build doc entries for the prompt
    doc_entries = []
    for document in query["relevance_documents"]:
        doc_id = document['doc_id']
        recipe_row = df_recipes_cleaned[df_recipes_cleaned["ID"] == doc_id]
        if recipe_row.empty:
            continue

        recipe_info = {}
        for col_key, col_info in COLUMNS_TEXT.items():
            recipe_info[col_info["start_text"]] = recipe_row.iloc[0][col_info["column_name"]]

        doc_entries.append({"doc_id": doc_id, "recipe_info": recipe_info})
    
    doc_entries_list = split_docs_recursively(client=SnowflakeClient(), model=LLM_MODEL, max_tokens=LLM_MODEL_CONTEXT_WINDOWS, base_template_size=num_token_prompt_template + num_token_doc, doc_entries=doc_entries)
    
    for doc_batch in doc_entries_list:
        prompt = build_prompt(prompt_template, query_text, doc_entries)

        # Call the LLM
        llm_response = get_llm_response(client=SnowflakeClient(), model=LLM_MODEL, prompt=prompt, response_format=json_schema, temperature=LLM_TEMPERATURE, max_tokens=LLM_MAX_TOKENS)

        # Extract and parse JSON (this now returns a proper dict, not a string)
        json_output = json.loads(llm_response)
        print(json_output)
    
        llm_results.append(json_output)
        
    print(f"✓ Parsed JSON for query: {query_text}")

1540
575
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
{'query_text': 'a vegan dessert that require minimum time to prepare', 'relevance_judgments': [{'doc_id': 515586, 'justification': 'The recipe is vegan, quick (blending time + freezing time), and a dessert. Fully satisfies all criteria.', 'relevance_score': 1}, {'doc_id': 33533, 'justification': "The recipe is a dessert and vegan, but the preparation time includes 30 minutes for freezing and 2 hours for total freezing time, which exceeds the 'minimum time' criteria slightly (2 notable differences).", 'relevance_score': 0.5}, {'doc_id': 501179, 'justification': "The recipe is a vegan dessert but it contains baking time. It does not meet the 'minimum time' criteria (contradicts the query).", 'relevance_score': 0}, {'doc_id': 33529, 'justification': "Not a dessert, is a vegan bread. Doesn't meet the dessert requirement. (Contradicts query)", 'relevance_score': 0}, {'doc_id': 501144, 'justification': 'The recipe is a vegan de

## compare LLM response to the ground truth ##

**compare the LLM response with the ground truth**

In [ ]:
coherence, coherence_per_query = compare_ground_truth_vs_llm(ground_truth_results, llm_results)

QUERY_LLM_RESULTS_PATH = config['query_llm_file_path'].format(
    experiment_id=EXPERIENCE_ID 
)

print("Final coherence score:", coherence)
print("Per-query coherence:", coherence_per_query)

llm_results.append({'COHERENCE_SCORE': coherence})

# Save results - now they're proper dicts, not strings
with open(QUERY_LLM_RESULTS_PATH, "w", encoding="utf-8") as f:
    json.dump(llm_results, f, indent=2, ensure_ascii=False)

Final coherence score: 0.75
Per-query coherence: [0.75]


**write the config file for that experience**

In [ ]:
OUPUT_EXPERIMENT_DIR = config["output_experiments_dir"].format(
    experiment_id=EXPERIENCE_ID 
)

# Write the config file
with open(os.path.join(OUPUT_EXPERIMENT_DIR, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4)